In [16]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch

# Set up S3 bucket and paths to input data and model artifacts
bucket_name = "store-objects-detection"
prefix = "sagemaker/SKU110K"

training_data_path = f's3://{bucket_name}/{prefix}'
model_artifacts_path = f's3://{bucket_name}/model-artifacts/'

# Set up SageMaker session, role, and instance types
sagemaker_session = sagemaker.Session()
train_instance_type = 'ml.g4dn.xlarge'

role = "sagemaker-iam-role"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [17]:
from sagemaker.image_uris import retrieve 
training_image = retrieve("pytorch", 
                            region="eu-central-1", 
                            version="1.13.1", 
                            py_version="py39", 
                            instance_type=train_instance_type, 
                            image_scope="training")
print(training_image)

763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:1.13.1-gpu-py39


In [18]:
# Define the training job
estimator = PyTorch(
    image_uri=training_image,
    source_dir="code",
    entry_point="yolo.py",
    role=role,
    py_version="py39",
    framework_version="1.13.1",
    instance_count=1,
    instance_type=train_instance_type,
    output_path=model_artifacts_path,
    sagemaker_session=sagemaker_session
)
estimator.fit({'train': f's3://{bucket_name}/{prefix}'}, logs="All")

INFO:sagemaker:Creating training-job with name: pytorch-training-2023-06-10-11-26-28-195


2023-06-10 11:26:35 Starting - Starting the training job...
2023-06-10 11:26:52 Starting - Preparing the instances for training..

KeyboardInterrupt: 